<a href="https://colab.research.google.com/github/funway/nid-imbalance-study/blob/main/utils/report_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 汇总报告文件, 生成对比表格

In [1]:
!pip install XlsxWriter

import os
import re
import pandas as pd

from pathlib import Path
from datetime import datetime
from google.colab import drive
from IPython.display import display
from xlsxwriter.utility import xl_col_to_name


## mount google drive
from google.colab import drive
drive.mount('/content/drive')


### Globals ###
## 数据文件目录
dataset = 'cse-cic-ids2018'
project_folder = Path('/content/drive/MyDrive/NYIT/880')
report_folder = project_folder / 'data/classification/reports'
compare_folder = project_folder / 'data/classification/compares'


# 二元组=(报告文件名的正则匹配规则, 输出的 compare 文件名)
# report_reg_and_output = (r'.*_report\.txt$', '所有报告')
# report_reg_and_output = (r'_s0_.*_report\.txt$', '不同scaling_s0_不做重采样')
# report_reg_and_output = (r'ros1\+cgan.*_report\.txt$', 'ros1+cgan的所有报告')
report_reg_and_output = (r'(?:_s0_|,gen\[128,).*_report\.txt$', '生成器噪声128的报告')

report_reg = report_reg_and_output[0]
output_path = compare_folder / f'{report_reg_and_output[1]}_{datetime.now().strftime("%m%d%H%M")}.xlsx'

# Excel 表格的单列最大宽度
MAX_COL_WIDTH = 70

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 8.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
if output_path.exists():
    print(f"\n[{datetime.now().strftime('%x %X')}] ⚠️ 已存在文件：{output_path}\n")
    choice = input("请输入 O（覆盖），R（重命名），或 C（取消保存）：").strip().upper()

    if choice == "O":
        # 用户选择覆盖，则不做任何修改，直接写
        print("💾 选择覆盖原文件。")
        pass
    elif choice == "R":
        # 用户选择重命名，则让用户再输入一个新文件名
        new_name = input("请输入新的文件名（.xlsx 后缀会自动添加，如果你不写也没关系）：").strip()
        if not new_name.lower().endswith(".xlsx"):
            new_name += ".xlsx"
        output_path = compare_folder / new_name
        print(f"✅ 新文件将保存为：{output_path}")
    else:
        # 用户选择取消
        print("❌ 已取消保存。")
        output_path = None

In [3]:
# 3. 确保 compare_folder 存在
compare_folder.mkdir(parents=True, exist_ok=True)

# 4. 定义需要提取的正则模式
patterns = {
    'Macro_Precision':    r"Macro average[\s\S]*?Precision:\s*([0-9]*\.[0-9]+)",
    'Macro_Recall':       r"Macro average[\s\S]*?Recall:\s*([0-9]*\.[0-9]+)",
    'Macro_F1':           r"Macro average[\s\S]*?F1 Score:\s*([0-9]*\.[0-9]+)",
    'Weighted_Precision': r"Weighted average[\s\S]*?Precision:\s*([0-9]*\.[0-9]+)",
    'Weighted_Recall':    r"Weighted average[\s\S]*?Recall:\s*([0-9]*\.[0-9]+)",
    'Weighted_F1':        r"Weighted average[\s\S]*?F1 Score:\s*([0-9]*\.[0-9]+)",
    'Accuracy':           r"Accuracy:\s*([0-9]*\.[0-9]+)",
    'Balanced_Accuracy':  r"Balanced Accuracy:\s*([0-9]*\.[0-9]+)"
}

# 5. 定义用于提取“最后五个 Binary Report 指标”的正则
binary_patterns = {
    'Binary_Accuracy':  r"Binary Report[\s\S]*?🎯 Accuracy\s*:\s*([0-9]*\.[0-9]+)",
    'Binary_Precision': r"Binary Report[\s\S]*?✅ Precision\s*:\s*([0-9]*\.[0-9]+)",
    'Binary_Recall':    r"Binary Report[\s\S]*?🔁 Recall / DR\s*:\s*([0-9]*\.[0-9]+)",
    'Binary_F1':        r"Binary Report[\s\S]*?🎯 F1 Score\s*:\s*([0-9]*\.[0-9]+)",
    'Binary_FPR':       r"Binary Report[\s\S]*?🚨 FPR.*?:\s*([0-9]*\.[0-9]+)"
}

# 6. 遍历 report_folder 中所有匹配 report_reg 的 .txt 文件
data_rows = []
for txt_path in report_folder.iterdir():
    if txt_path.is_file() and re.search(report_reg, txt_path.name):
        content = txt_path.read_text(encoding='utf-8', errors='ignore')

        row = {'Filename': txt_path.name}

        # 抽取宏平均 + 加权平均 + 全局 Accuracy / Balanced Accuracy
        for key, pat in patterns.items():
            match = re.search(pat, content, flags=re.MULTILINE)
            row[key] = float(match.group(1)) if match else None

        # 抽取“Binary Report”最后五个指标
        for key, bpat in binary_patterns.items():
            bmatch = re.search(bpat, content)
            row[key] = float(bmatch.group(1)) if bmatch else None

        data_rows.append(row)

# 7. 构造 DataFrame, 并且按 Filename 排序
df_metrics = pd.DataFrame(data_rows, columns=[
    'Filename',
    'Macro_Precision', 'Macro_Recall',   'Macro_F1',
    'Weighted_Precision', 'Weighted_Recall', 'Weighted_F1',
    'Accuracy', 'Balanced_Accuracy',
    'Binary_Accuracy', 'Binary_Precision', 'Binary_Recall', 'Binary_F1', 'Binary_FPR'
])
df_metrics.sort_values('Filename', inplace=True)

# 8. 在 Colab 中展示生成的 DataFrame
display(df_metrics)

# 9. 将结果保存为 Excel 文件到 compare_folder
# 使用 ExcelWriter + xlsxwriter 将 DataFrame 写入 Excel，
# 同时自动调整列宽并把所有数值格式为小数点后 6 位
if output_path:
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        df_metrics.to_excel(writer, sheet_name='Sheet1', index=False)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']

        # 定义数值格式：6 位小数
        num_format = workbook.add_format({'num_format': '0.000000'})
        # 定义红色格式，用于标红最大值
        red_fmt = workbook.add_format({'bg_color': '#EBF1DE', 'font_color': '#FF0000'})

        n_rows, n_cols = df_metrics.shape

        # 遍历每一列(idx 返回列 id, col 返回列名)，先计算该列最大字符串长度，用以自动设置列宽
        for idx, col in enumerate(df_metrics.columns):
            # 计算该列所有 cell 的字符串长度
            str_lengths = df_metrics[col].fillna('').astype(str).map(len)  # 注意先把 NaN 转成空字符串，否则 len("nan")=3
            # 与列名长度进行 max 取值
            max_length = max(str_lengths.max(), len(col)) + 1  # +1 作为缓冲
            # 再限制最大长度
            max_length = min(max_length, MAX_COL_WIDTH)

            if idx == 0:
                # 如果是第 0 列（Filename 列），只设宽度，不设置数值格式
                worksheet.set_column(idx, idx, max_length)
            else:
                # 其他列既调整列宽，又应用小数点后 6 位格式
                worksheet.set_column(idx, idx, max_length, num_format)

                # 在这一列上添加条件格式：Top 1 即标红该列最大值
                col_letter = xl_col_to_name(idx)  # 将索引转换为 Excel 列名，比如 0 -> 'A'
                start_row, end_row = 2, n_rows + 1
                cell_range = f'{col_letter}{start_row}:{col_letter}{end_row}' # 要比较大小的 cell 坐标范围
                format_type = 'bottom' if idx==(n_cols-1) else 'top'  # 只有最后一列是标红最小值
                worksheet.conditional_format(cell_range, { 'type': format_type,
                                                           'value': 1,
                                                           'format': red_fmt}
                                             )

    print(f"[{datetime.now().strftime('%x %X')}] 💾 已将提取的指标保存到：{output_path}")


,Filename,Macro_Precision,Macro_Recall,Macro_F1,Weighted_Precision,Weighted_Recall,Weighted_F1,Accuracy,Balanced_Accuracy,Binary_Accuracy,Binary_Precision,Binary_Recall,Binary_F1,Binary_FPR
18,cse-cic-ids2018_l1pminmax_s0_NA_NA_CNN-best_re...,0.894998,0.835302,0.850318,0.943943,0.949997,0.942426,0.949997,0.835302,0.969688,0.992560,0.954775,0.973301,0.009830
19,cse-cic-ids2018_l1pminmax_s0_NA_NA_CNN-final_r...,0.894994,0.835298,0.850312,0.943939,0.949995,0.942423,0.949995,0.835298,0.969686,0.992560,0.954771,0.973299,0.009830
11,cse-cic-ids2018_l1pminmax_s0_NA_NA_MLP-best_re...,0.814177,0.752855,0.759754,0.943787,0.949797,0.942021,0.949797,0.752855,0.969448,0.992307,0.954604,0.973090,0.010165
13,cse-cic-ids2018_l1pminmax_s0_NA_NA_MLP-final_r...,0.806040,0.774840,0.774050,0.943780,0.949772,0.941990,0.949772,0.774840,0.969441,0.992284,0.954615,0.973085,0.010195
12,cse-cic-ids2018_l1pminmax_s0_NA_NA_RF_report.txt,0.712723,0.696091,0.691181,0.937216,0.944219,0.934705,0.944219,0.696091,0.964137,0.990404,0.947203,0.968322,0.012605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,"cse-cic-ids2018_standard_s3_ros1+cgan-m(n128,f...",0.832260,0.866500,0.841121,0.941513,0.942019,0.940702,0.942019,0.866500,0.961299,0.968028,0.964994,0.966508,0.043775
204,"cse-cic-ids2018_standard_s4_ros1+cgan-b(n128,f...",0.843189,0.842416,0.829834,0.944323,0.950233,0.942471,0.950233,0.842416,0.969557,0.992442,0.954662,0.973186,0.009985
207,"cse-cic-ids2018_standard_s4_ros1+cgan-b(n128,f...",0.874166,0.859524,0.849346,0.944567,0.949759,0.939728,0.949759,0.859524,0.969273,0.995346,0.951350,0.972850,0.006110
196,"cse-cic-ids2018_standard_s6_ros1+cgan-b(n128,f...",0.843305,0.853965,0.830782,0.944217,0.950143,0.942890,0.950143,0.853965,0.969380,0.991213,0.955558,0.973059,0.011635


[06/25/25 06:06:01] 💾 已将提取的指标保存到：/content/drive/MyDrive/NYIT/880/data/classification/compares/生成器噪声128的报告_06250605.xlsx


In [4]:
# from google.colab import runtime
# print(f"[{datetime.now().strftime('%x %X')}] ⛔ 运行结束. shutdown now...")
# runtime.unassign()